<a href="https://colab.research.google.com/github/cerebraters/datasets_text/blob/main/TESTpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.7 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import json
from datetime import datetime, timedelta
import joblib
import os
import sys
import warnings
from collections import defaultdict, Counter
from typing import List, Dict, Any, Optional, Union, Tuple
from functools import lru_cache
import heapq
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [8]:
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [9]:
class ItemPredictor:
    def __init__(self, min_cycles: int = 1, period_days: int = 30, top_k: int = 3,
                 recent_weight: float = 2.0, decay_factor: float = 0.95,
                 diversity_factor: float = 0.3, popularity_penalty: float = 0.7):
        self.min_cycles = min_cycles
        self.period_days = period_days
        self.top_k = top_k
        self.recent_weight = recent_weight
        self.decay_factor = decay_factor
        self.diversity_factor = diversity_factor
        self.popularity_penalty = popularity_penalty
        self.user_history = defaultdict(list)
        self.global_item_freq = Counter()
        self.global_top_items = []
        self.update_counter = 0
        self.max_history_size = 200
        self.item_decay_rates = defaultdict(float)

    def update_history(self, user_id: Union[str, int], item: str, purchase_date: datetime) -> None:
        user_id = str(user_id)
        self.user_history[user_id].append((item, purchase_date))
        self.global_item_freq[item] += 1

        # Apply decay to old frequencies
        for item in list(self.global_item_freq.keys()):
            self.global_item_freq[item] *= self.decay_factor

        if len(self.user_history[user_id]) > self.max_history_size:
            self.user_history[user_id] = sorted(
                self.user_history[user_id][-self.max_history_size:],
                key=lambda x: x[1], reverse=True
            )

        self.update_counter += 1
        if self.update_counter % 50 == 0:
            self.calculate_global_top_items()

    def calculate_global_top_items(self) -> None:
        if self.global_item_freq:
            median_freq = np.median(list(self.global_item_freq.values()))
            adaptive_min_cycles = max(1, int(median_freq * 0.3))

            cyclic_items = {item: count for item, count in self.global_item_freq.items()
                          if count >= adaptive_min_cycles}

            if cyclic_items:
                self.global_top_items = [item for item, _ in
                                       Counter(cyclic_items).most_common(self.top_k*5)]
            else:
                self.global_top_items = []

    def predict_for_user(self, user_id: Union[str, int], current_date: Optional[datetime] = None) -> List[str]:
        current_date = current_date or datetime.now()
        user_id = str(user_id)
        predictions = []
        user_items = Counter()

        if user_id in self.user_history:
            item_weights = defaultdict(float)
            user_items = Counter(item for item, _ in self.user_history[user_id])

            user_min_cycles = max(1, int(len(self.user_history[user_id]) * 0.1))
            cyclic_items = {item for item, count in user_items.items()
                           if count >= max(self.min_cycles, user_min_cycles)}

            for item, purchase_date in sorted(self.user_history[user_id], key=lambda x: x[1], reverse=True):
                if item not in cyclic_items:
                    continue

                days_ago = (current_date - purchase_date).days
                if days_ago > self.period_days:
                    continue

                recency_weight = self.recent_weight if days_ago <= 7 else 1.0
                frequency_weight = np.log1p(user_items[item])
                decay_weight = self.decay_factor ** days_ago

                # Apply popularity penalty for top global items
                base_weight = recency_weight * frequency_weight * decay_weight
                if item in self.global_top_items[:10]:
                    base_weight *= self.popularity_penalty

                item_weights[item] += base_weight * (1 + random.uniform(-self.diversity_factor, self.diversity_factor))

            if item_weights:
                predictions = [item for item, _ in sorted(
                    item_weights.items(),
                    key=lambda x: (-x[1], x[0])
                )[:self.top_k*3]]

        # Fallback with diversity
        fallback_items = []
        for item in self.global_top_items:
            weight = user_items[item] * 2 if item in user_items else 1
            if item in self.global_top_items[:10]:
                weight *= 0.8  # Additional penalty for top global items
            fallback_items.append((item, weight))

        fallback_items = [item for item, _ in sorted(fallback_items, key=lambda x: -x[1])[:self.top_k*3]]

        # Final selection with enforced diversity
        all_candidates = list(set(predictions + fallback_items))
        if len(all_candidates) >= 3:
            # Select 1-2 personalized items and 1-2 diverse items
            personalized = [i for i in predictions if i in user_items][:2]
            diverse_pool = [i for i in all_candidates if i not in personalized]
            diverse = random.sample(diverse_pool, min(len(diverse_pool), self.top_k - len(personalized)))
            return personalized + diverse

        return all_candidates[:self.top_k]


In [10]:
class BasketPredictor:
    def __init__(self, min_samples: int = 1, top_k: int = 5, min_unique_checks: int = 1,
                 diversity_factor: float = 0.3, place_diversity: float = 0.3):
        self.min_samples = min_samples
        self.top_k = top_k
        self.min_unique_checks = min_unique_checks
        self.diversity_factor = diversity_factor
        self.place_diversity = place_diversity
        self.item_encoder = LabelEncoder()
        self.feature_encoder = ColumnTransformer(
            transformers=[
                ('customer', OneHotEncoder(handle_unknown='ignore'), ['customerId']),
                ('place', OneHotEncoder(handle_unknown='ignore'), ['pointName']),
                ('time', OneHotEncoder(handle_unknown='ignore'), ['day_of_week', 'month', 'time_of_day'])
            ],
            remainder='drop'
        )
        self.model = None
        self.context_recommendations = defaultdict(Counter)
        self.general_top_items = []
        self.mlb = MultiLabelBinarizer()
        self.user_item_preferences = defaultdict(Counter)

    def prepare_data(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
        if df.empty:
            return pd.DataFrame(), pd.DataFrame()

        item_stats = df.groupby('goodsName').agg(
            total_purchases=('goodsName', 'count'),
            unique_checks=('timeCheck', 'nunique'),
            unique_customers=('customerId', 'nunique')
        )

        adaptive_min_samples = max(1, int(len(df) * 0.001))
        adaptive_min_checks = max(1, int(df['timeCheck'].nunique() * 0.01))

        valid_items = item_stats[
            (item_stats['total_purchases'] >= max(self.min_samples, adaptive_min_samples)) &
            (item_stats['unique_checks'] >= max(self.min_unique_checks, adaptive_min_checks))
        ].index.tolist()

        if not valid_items:
            return pd.DataFrame(), pd.DataFrame()

        basket_df = df[df['goodsName'].isin(valid_items)] \
            .groupby(['customerId', 'timeCheck', 'pointName', 'day_of_week', 'month', 'time_of_day'])['goodsName'] \
            .apply(list).reset_index(name='basket')

        for _, row in basket_df.iterrows():
            self.user_item_preferences[row['customerId']].update(row['basket'])

        self.item_encoder.fit(valid_items)
        y = basket_df['basket'].apply(lambda x: [i for i in x if i in valid_items])
        valid_indices = y.apply(len) > 0
        basket_df = basket_df[valid_indices]
        y = y[valid_indices]

        self.mlb.fit([self.item_encoder.classes_])
        y_encoded = self.mlb.transform(y)

        X = basket_df[['customerId', 'pointName', 'day_of_week', 'month', 'time_of_day']]

        return X, pd.DataFrame(y_encoded, columns=self.item_encoder.classes_)

    def train(self, df: pd.DataFrame) -> None:
        for _, row in df.iterrows():
            key = (row['pointName'], row['day_of_week'], row['time_of_day'])
            self.context_recommendations[key][row['goodsName']] += 1

        item_stats = df.groupby('goodsName').agg(
            total_purchases=('goodsName', 'count'),
            unique_checks=('timeCheck', 'nunique')
        )

        adaptive_min_checks = max(1, int(df['timeCheck'].nunique() * 0.01))
        cyclic_items = item_stats[item_stats['unique_checks'] >= adaptive_min_checks].index

        self.general_top_items = df[df['goodsName'].isin(cyclic_items)]['goodsName'] \
            .value_counts().head(self.top_k*5).index.tolist()

        X, y = self.prepare_data(df)

        if X.empty or y.empty:
            return

        pipeline = Pipeline([
            ('preprocessor', self.feature_encoder),
            ('classifier', OneVsRestClassifier(
                CatBoostClassifier(
                    iterations=200,
                    depth=6,
                    learning_rate=0.1,
                    loss_function='MultiClass',
                    verbose=False,
                    random_state=42,
                    thread_count=-1
                ), n_jobs=-1
            ))
        ])

        pipeline.fit(X, y)
        self.model = pipeline

    def predict(self, context: Dict) -> List[str]:
        try:
            customer_id = context['customerId']

            # Personalized recommendations with diversity
            if customer_id in self.user_item_preferences:
                user_items = self.user_item_preferences[customer_id]
                user_top = [item for item, _ in user_items.most_common(self.top_k*3)]

                if len(user_top) >= 3:
                    # Select 1-2 top items and 1-2 less frequent
                    selected = user_top[:1]
                    diverse_pool = [i for i in user_top if i not in selected]
                    if diverse_pool:
                        selected += random.sample(diverse_pool, min(2, len(diverse_pool)))
                    return selected[:self.top_k]

            # Model predictions with diversity
            if self.model is not None:
                X_pred = pd.DataFrame([{
                    'customerId': context['customerId'],
                    'pointName': context['pointName'],
                    'day_of_week': context['day_of_week'],
                    'month': context['month'],
                    'time_of_day': context['time_of_day']
                }])

                if hasattr(self.model, 'predict_proba'):
                    probs = self.model.predict_proba(X_pred)
                    scores = np.array([p[:, 1] if p.ndim > 1 else p for p in probs]).squeeze()
                else:
                    scores = self.model.decision_function(X_pred).squeeze()

                if scores.ndim > 0:
                    top_indices = np.argsort(-scores)[:self.top_k*5]
                    candidates = [self.item_encoder.classes_[i] for i in top_indices
                                if i < len(self.item_encoder.classes_)]

                    if customer_id in self.user_item_preferences:
                        user_prefs = self.user_item_preferences[customer_id]
                        candidates = sorted(candidates, key=lambda x: -user_prefs.get(x, 0))

                    # Enforce diversity in final selection
                    if len(candidates) >= 3:
                        selected = candidates[:1]
                        diverse_pool = [i for i in candidates if i not in selected]
                        if diverse_pool:
                            selected += random.sample(diverse_pool, min(2, len(diverse_pool)))
                        return selected[:self.top_k]

            # Fallback with context and diversity
            key = (context['pointName'], context['day_of_week'], context['time_of_day'])
            items = [item for item, _ in self.context_recommendations.get(key, Counter()).most_common(self.top_k*3)]

            if len(items) >= 3:
                selected = items[:1]
                diverse_pool = [i for i in items if i not in selected]
                if diverse_pool:
                    selected += random.sample(diverse_pool, min(2, len(diverse_pool)))
                return selected[:self.top_k]

            return items[:self.top_k] if items else self.general_top_items[:self.top_k]

        except Exception as e:
            print(f"Prediction error: {str(e)}", file=sys.stderr)
            return self._get_fallback_recommendations(context)

    def _get_fallback_recommendations(self, context: Dict) -> List[str]:
        key = (context['pointName'], context['day_of_week'], context['time_of_day'])
        items = [item for item, _ in self.context_recommendations.get(key, Counter()).most_common(self.top_k)]
        return items[:self.top_k] if items else self.general_top_items[:self.top_k]



In [11]:
class PurchasePredictor:
    def __init__(self):
        self.time_model = None
        self.place_model = None
        self.item_predictor = ItemPredictor(
            min_cycles=1,
            recent_weight=2.0,
            diversity_factor=0.3,
            popularity_penalty=0.7
        )
        self.basket_predictor = BasketPredictor(
            min_unique_checks=1,
            diversity_factor=0.3,
            place_diversity=0.3
        )
        self.last_trained = None
        self.general_top_items = []
        self.general_common_place = ""
        self.general_common_hour = None
        self.min_samples = 2
        self.trained_customers = set()
        self.user_time_preferences = defaultdict(list)
        self.user_place_preferences = defaultdict(list)
        self.user_purchase_hours = defaultdict(list)
        self.user_visit_places = defaultdict(list)
        self.user_purchase_intervals = defaultdict(list)
        self.user_last_purchase = defaultdict(datetime)

    def _get_time_of_day(self, hour: int) -> str:
        if 5 <= hour < 11: return 'morning'
        elif 11 <= hour < 16: return 'afternoon'
        elif 16 <= hour < 21: return 'evening'
        else: return 'night'

    def _analyze_user_behavior(self, df: pd.DataFrame):
        for customer_id, group in df.groupby('customerId'):
            customer_id = str(customer_id)
            self.user_purchase_hours[customer_id] = group['hour'].value_counts().index.tolist()
            self.user_visit_places[customer_id] = group['pointName'].value_counts().index.tolist()

            if len(group) > 1:
                sorted_dates = group['datetime'].sort_values()
                intervals = (sorted_dates.diff().dt.total_seconds() / 3600).dropna()
                self.user_purchase_intervals[customer_id] = intervals.tolist()

            self.user_last_purchase[customer_id] = group['datetime'].max()

            if len(group) >= 2:
                hour_mode = group['hour'].mode()
                if not hour_mode.empty:
                    self.user_time_preferences[customer_id] = hour_mode.tolist()

            place_mode = group['pointName'].mode()
            if not place_mode.empty:
                self.user_place_preferences[customer_id] = place_mode.tolist()

    def load_and_preprocess(self, json_data: List[Dict]) -> pd.DataFrame:
        try:
            df = pd.DataFrame(json_data)
            required_columns = ['customerId', 'timeCheck', 'pointName', 'goodsName']

            for col in required_columns:
                if col not in df.columns:
                    raise ValueError(f"Missing required column: {col}")

            df = df.drop_duplicates(subset=required_columns)
            df['pointName'] = df['pointName'].str.strip()
            df['goodsName'] = df['goodsName'].str.strip()

            df['datetime'] = pd.to_datetime(df['timeCheck'])
            df['hour'] = df['datetime'].dt.hour
            df['day_of_week'] = df['datetime'].dt.dayofweek
            df['month'] = df['datetime'].dt.month
            df['time_of_day'] = df['hour'].apply(self._get_time_of_day)

            df = df.dropna(subset=required_columns)
            self._analyze_user_behavior(df)

            for _, row in df.iterrows():
                self.item_predictor.update_history(
                    str(row['customerId']),
                    row['goodsName'],
                    row['datetime']
                )

            if not df.empty:
                item_stats = df.groupby('goodsName').agg(
                    total_purchases=('goodsName', 'count'),
                    unique_checks=('timeCheck', 'nunique'),
                    unique_customers=('customerId', 'nunique')
                )

                adaptive_min_checks = max(1, int(df['timeCheck'].nunique() * 0.01))
                cyclic_items = item_stats[item_stats['unique_checks'] >= adaptive_min_checks].index

                self.general_top_items = df[df['goodsName'].isin(cyclic_items)] \
                    ['goodsName'].value_counts().head(15).index.tolist()
                self.general_common_place = df['pointName'].mode()[0] if not df['pointName'].empty else ""
                self.general_common_hour = df['hour'].mode()[0] if not df['hour'].empty else None

            return df.drop(columns=['datetime'])
        except Exception as e:
            print(f"Data processing error: {e}", file=sys.stderr)
            return pd.DataFrame()

    def train_models(self, df: pd.DataFrame) -> None:
        if df.empty:
            print("No data for training", file=sys.stderr)
            return

        print("\nData analysis before training:")
        print("Unique customers:", df['customerId'].nunique())
        print("Unique places:", df['pointName'].nunique())
        print("Unique hours:", df['hour'].nunique())
        print("Average places per customer:",
              df.groupby('customerId')['pointName'].nunique().mean())

        item_stats = df.groupby('goodsName').agg(
            total_purchases=('goodsName', 'count'),
            unique_checks=('timeCheck', 'nunique'),
            unique_customers=('customerId', 'nunique')
        )

        adaptive_min_samples = max(1, int(len(df) * 0.001))
        adaptive_min_checks = max(1, int(df['timeCheck'].nunique() * 0.01))

        valid_items = item_stats[
            (item_stats['total_purchases'] >= max(self.min_samples, adaptive_min_samples)) &
            (item_stats['unique_checks'] >= adaptive_min_checks)
        ].index

        df_filtered = df[df['goodsName'].isin(valid_items)]

        if df_filtered.empty:
            print("Warning: No items with cyclic purchases", file=sys.stderr)
            return

        self.trained_customers = set(df_filtered['customerId'].astype(str).unique())

        # Time model
        X_time = df_filtered[['customerId', 'day_of_week', 'month', 'time_of_day']]
        y_time = df_filtered['hour']

        if not X_time.empty and y_time.nunique() > 1:
            self.time_model = Pipeline([
                ('preprocessor', ColumnTransformer(
                    transformers=[
                        ('customer', OneHotEncoder(handle_unknown='ignore'), ['customerId']),
                        ('time', OneHotEncoder(handle_unknown='ignore'),
                         ['day_of_week', 'month', 'time_of_day'])
                    ]
                )),
                ('regressor', RandomForestRegressor(
                    n_estimators=100,
                    random_state=42,
                    n_jobs=-1,
                    min_samples_leaf=3
                ))
            ])
            self.time_model.fit(X_time, y_time)
            print("\nTime model trained with user preferences")

        # Place model with LGBM
        X_place = df_filtered[['customerId', 'day_of_week', 'month', 'time_of_day', 'hour']]
        y_place = df_filtered['pointName']

        if not X_place.empty and y_place.nunique() > 1:
            self.place_model = Pipeline([
                ('preprocessor', ColumnTransformer(
                    transformers=[
                        ('customer', OneHotEncoder(handle_unknown='ignore'), ['customerId']),
                        ('time', OneHotEncoder(handle_unknown='ignore'),
                         ['day_of_week', 'month', 'time_of_day']),
                        ('hour', OneHotEncoder(handle_unknown='ignore'), ['hour'])
                    ]
                )),
                ('classifier', LGBMClassifier(
                    n_estimators=100,
                    random_state=42,
                    n_jobs=-1,
                    class_weight='balanced',
                    min_child_samples=3
                ))
            ])
            self.place_model.fit(X_place, y_place)
            print("Place model trained with additional features")

        # Train basket model
        print("\nTraining basket model...")
        self.basket_predictor.train(df_filtered)

        self.last_trained = datetime.now()
        print("\nTraining completed")

    def predict_for_customer(self, customer_id: Union[str, int],
                           day_of_week: Optional[int] = None,
                           month: Optional[int] = None) -> Dict[str, Any]:
        customer_id = str(customer_id)
        current_date = datetime.now()
        last_purchase = self.user_last_purchase.get(customer_id)

        if day_of_week is None:
            day_of_week = current_date.weekday()
        if month is None:
            month = current_date.month

        current_hour = current_date.hour
        time_of_day = self._get_time_of_day(current_hour)

        # User status detection
        is_new_user = customer_id not in self.trained_customers
        is_inactive = False

        if last_purchase:
            days_inactive = (current_date - last_purchase).days
            is_inactive = days_inactive > 30

        # Get user preferences
        user_hours = self.user_purchase_hours.get(customer_id, [])
        user_places = self.user_visit_places.get(customer_id, [])
        user_intervals = self.user_purchase_intervals.get(customer_id, [])

        # Time prediction with diversity
        predicted_hour = user_hours[0] if user_hours else self.general_common_hour or current_hour

        if not is_new_user and self.time_model:
            try:
                time_input = pd.DataFrame([{
                    'customerId': customer_id,
                    'day_of_week': day_of_week,
                    'month': month,
                    'time_of_day': time_of_day
                }])

                model_pred = self.time_model.predict(time_input)[0]

                if user_hours:
                    user_avg = np.mean(user_hours[:3])
                    predicted_hour = int(round(model_pred * 0.6 + user_avg * 0.4))
                else:
                    predicted_hour = int(round(model_pred))

                predicted_hour = max(0, min(23, predicted_hour))

                if user_intervals and not is_inactive:
                    avg_interval = np.mean(user_intervals)
                    if avg_interval > 0:
                        expected_hour = last_purchase.hour + (avg_interval % 24)
                        predicted_hour = int(round(predicted_hour * 0.7 + expected_hour * 0.3))
            except Exception as e:
                print(f"Time prediction error: {e}", file=sys.stderr)

        # Place prediction with enforced diversity
        predicted_place = user_places[0] if user_places else self.general_common_place

        if not is_new_user and self.place_model:
            try:
                place_input = pd.DataFrame([{
                    'customerId': customer_id,
                    'day_of_week': day_of_week,
                    'month': month,
                    'time_of_day': time_of_day,
                    'hour': predicted_hour
                }])

                if len(user_places) >= 2:
                    place_probs = self.place_model.predict_proba(place_input)
                    top_places = self.place_model.classes_[np.argsort(-place_probs[0])][:5]

                    # Add randomness to place selection
                    if random.random() < self.basket_predictor.place_diversity:
                        predicted_place = random.choice(top_places[:3])
                    else:
                        for place in top_places:
                            if place in user_places[:3]:
                                predicted_place = place
                                break
                        else:
                            predicted_place = top_places[0]
                else:
                    predicted_place = self.place_model.predict(place_input)[0]
            except Exception as e:
                print(f"Place prediction error: {e}", file=sys.stderr)

        # Prepare basket context
        basket_context = {
            'customerId': customer_id,
            'pointName': predicted_place,
            'day_of_week': day_of_week,
            'month': month,
            'time_of_day': time_of_day,
            'is_new_user': is_new_user or is_inactive
        }

        # Get basket predictions
        top_items = self.basket_predictor.predict(basket_context)[:3]

        # For new/inactive users, add more diversity
        if is_new_user or is_inactive:
            if len(top_items) > 1 and random.random() < 0.5:
                top_items[-1] = random.choice(self.general_top_items[:20])

        # Generate message
        message = None
        if is_new_user:
            purchase_count = len(self.item_predictor.user_history.get(customer_id, []))
            if purchase_count < 2:
                message = f"Make {2 - purchase_count} more purchases to get personalized recommendations"
        elif is_inactive:
            days = (current_date - last_purchase).days
            message = f"We've missed you! It's been {days} days since your last purchase"

        # Create time window with variation
        window_size = random.choice([2, 3, 4])
        time_window_start = predicted_hour
        time_window_end = (predicted_hour + window_size) % 24
        time_window = f"{time_window_start:02d}:00-{time_window_end:02d}:00"

        return {
            'customerId': customer_id,
            'top_items': top_items,
            'predicted_time': time_window,
            'predicted_place': predicted_place,
            'is_new_user': is_new_user,
            'is_inactive': is_inactive,
            'message': message
        }

    def save_model(self, path: str) -> None:
        model_state = {
            'time_model': self.time_model,
            'place_model': self.place_model,
            'general_top_items': self.general_top_items,
            'general_common_place': self.general_common_place,
            'general_common_hour': self.general_common_hour,
            'last_trained': self.last_trained,
            'min_samples': self.min_samples,
            'trained_customers': list(self.trained_customers),
            'basket_predictor': self.basket_predictor,
            'item_predictor': {
                'user_history': {
                    user_id: [(item, date.isoformat()) for item, date in entries]
                    for user_id, entries in self.item_predictor.user_history.items()
                },
                'global_item_freq': dict(self.item_predictor.global_item_freq),
                'global_top_items': self.item_predictor.global_top_items,
                'update_counter': self.item_predictor.update_counter,
                'params': {
                    'min_cycles': self.item_predictor.min_cycles,
                    'period_days': self.item_predictor.period_days,
                    'top_k': self.item_predictor.top_k,
                    'recent_weight': self.item_predictor.recent_weight,
                    'decay_factor': self.item_predictor.decay_factor,
                    'diversity_factor': self.item_predictor.diversity_factor,
                    'popularity_penalty': self.item_predictor.popularity_penalty
                }
            },
            'user_behavior': {
                'time_preferences': dict(self.user_time_preferences),
                'place_preferences': dict(self.user_place_preferences),
                'purchase_hours': dict(self.user_purchase_hours),
                'visit_places': dict(self.user_visit_places),
                'purchase_intervals': dict(self.user_purchase_intervals),
                'last_purchase': {k: v.isoformat() for k, v in self.user_last_purchase.items()}
            }
        }

        joblib.dump(model_state, path)
        print(f"Model successfully saved to {path}")

    def load_model(self, path: str) -> None:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Model file {path} not found")

        model_state = joblib.load(path)

        self.time_model = model_state.get('time_model')
        self.place_model = model_state.get('place_model')
        self.general_top_items = model_state.get('general_top_items', [])
        self.general_common_place = model_state.get('general_common_place', "")
        self.general_common_hour = model_state.get('general_common_hour')
        self.last_trained = model_state.get('last_trained')
        self.min_samples = model_state.get('min_samples', 2)
        self.trained_customers = set(model_state.get('trained_customers', []))
        self.basket_predictor = model_state.get('basket_predictor', BasketPredictor())

        item_data = model_state.get('item_predictor', {})
        params = item_data.get('params', {})
        self.item_predictor = ItemPredictor(
            min_cycles=params.get('min_cycles', 1),
            period_days=params.get('period_days', 30),
            top_k=params.get('top_k', 3),
            recent_weight=params.get('recent_weight', 2.0),
            decay_factor=params.get('decay_factor', 0.95),
            diversity_factor=params.get('diversity_factor', 0.3),
            popularity_penalty=params.get('popularity_penalty', 0.7)
        )

        self.item_predictor.user_history = defaultdict(list)
        for user_id, entries in item_data.get('user_history', {}).items():
            for item, date_str in entries:
                self.item_predictor.user_history[user_id].append(
                    (item, datetime.fromisoformat(date_str)))

        self.item_predictor.global_item_freq = Counter(item_data.get('global_item_freq', {}))
        self.item_predictor.global_top_items = item_data.get('global_top_items', [])
        self.item_predictor.update_counter = item_data.get('update_counter', 0)

        behavior_data = model_state.get('user_behavior', {})
        self.user_time_preferences = defaultdict(list, behavior_data.get('time_preferences', {}))
        self.user_place_preferences = defaultdict(list, behavior_data.get('place_preferences', {}))
        self.user_purchase_hours = defaultdict(list, behavior_data.get('purchase_hours', {}))
        self.user_visit_places = defaultdict(list, behavior_data.get('visit_places', {}))
        self.user_purchase_intervals = defaultdict(list, behavior_data.get('purchase_intervals', {}))
        self.user_last_purchase = defaultdict(datetime)
        for user_id, date_str in behavior_data.get('last_purchase', {}).items():
            self.user_last_purchase[user_id] = datetime.fromisoformat(date_str)

        print(f"Model successfully loaded from {path}")

In [ ]:
def main_workflow(input_json_path: str, output_json_path: Optional[str] = None) -> List[Dict]:
    try:
        with open(input_json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
        print(f"Successfully loaded {len(json_data)} receipts")
    except Exception as e:
        print(f"Error loading JSON: {e}", file=sys.stderr)
        return []

    predictor = PurchasePredictor()
    try:
        df = predictor.load_and_preprocess(json_data)
        if df.empty:
            print("Error: No valid data", file=sys.stderr)
            return []

        predictor.train_models(df)
    except Exception as e:
        print(f"Training error: {e}", file=sys.stderr)
        return []

    predictions = []
    unique_customers = df['customerId'].astype(str).unique() if not df.empty else []
    print(f"\nFound {len(unique_customers)} customers")

    for customer_id in unique_customers:
        try:
            prediction = predictor.predict_for_customer(customer_id)
            predictions.append(prediction)
        except Exception as e:
            print(f"Prediction error for {customer_id}: {e}", file=sys.stderr)

    # Analyze prediction diversity
    unique_time_place = len(set((p['predicted_time'], p['predicted_place']) for p in predictions))
    all_items = [item for p in predictions for item in p['top_items']]
    unique_items = len(set(all_items))
    item_counts = Counter(all_items)

    print(f"\nPrediction diversity analysis:")
    print(f"- Unique time/place combinations: {unique_time_place} of {len(predictions)}")
    print(f"- Unique recommended items: {unique_items}")
    print(f"- Average recommendations per item: {len(predictions)*3/unique_items:.1f}")
    print(f"- Most recommended items:")
    for item, count in item_counts.most_common(5):
        print(f"  {item}: {count} times ({(count/len(predictions)*100):.1f}%)")

    if output_json_path and predictions:
        try:
            with open(output_json_path, 'w', encoding='utf-8') as f:
                json.dump(predictions, f, ensure_ascii=False, indent=2)
            print(f"\nPredictions saved to {output_json_path}")
        except Exception as e:
            print(f"Error saving predictions: {e}", file=sys.stderr)

    try:
        model_path = os.path.join(
            os.path.dirname(input_json_path) or os.getcwd(),
            'purchase_predictor_model.joblib'
        )
        predictor.save_model(model_path)
        print(f"Model saved to {model_path}")
    except Exception as e:
        print(f"Error saving model: {e}", file=sys.stderr)

    return predictions

if __name__ == "__main__":
    input_path = "/content/drive/MyDrive/ai.json"  # Update with your path
    output_path = "/content/drive/MyDrive/output.json"  # Update with your path

    predictions = main_workflow(
        input_json_path=input_path,
        output_json_path=output_path
    )

    if predictions:
        print("\nPrediction examples:")
        for i, pred in enumerate(predictions[:5]):
            print(f"\nPrediction #{i+1} for customer {pred['customerId']}:")
            print(f"- Top items: {', '.join(pred['top_items'])}")
            print(f"- Time window: {pred['predicted_time']}")
            print(f"- Place: {pred['predicted_place']}")
            if pred.get('message'):
                print(f"- Message: {pred['message']}")
            print(f"- Status: {'New' if pred['is_new_user'] else 'Inactive' if pred['is_inactive'] else 'Active'}")

Successfully loaded 40000 receipts

Data analysis before training:
Unique customers: 6094
Unique places: 67
Unique hours: 16
Average places per customer: 1.039875287167706


In [ ]:
from google.colab import files
import joblib

# Загружаем файл модели
uploaded = files.upload()
model_path = next(iter(uploaded.keys()))  # Получаем имя файла

# Загружаем модель
try:
    predictor = PurchasePredictor()
    predictor.load_model(model_path)
    print("✅ Модель успешно загружена!")
except Exception as e:
    print(f"❌ Ошибка загрузки модели: {e}")

In [ ]:
# Загружаем JSON с новыми чеками
new_data_upload = files.upload()
new_data_path = next(iter(new_data_upload.keys()))

# Читаем данные
with open(new_data_path, 'r', encoding='utf-8') as f:
    new_checks = json.load(f)

# Предобработка (как в load_and_preprocess)
df_new = predictor.load_and_preprocess(new_checks)

In [ ]:
predictions = {}
for user_id in df_new['customerId'].unique():
    try:
        # Явно конвертируем user_id в строку
        user_id_str = str(user_id)
        pred = predictor.predict_for_customer(user_id_str)

        # Конвертируем numpy.str_ в обычные строки
        pred['top_items'] = [str(item) for item in pred['top_items']]

        predictions[user_id_str] = pred
        print(f"Пользователь {user_id_str}: {pred['top_items']}")
    except Exception as e:
        print(f"Ошибка для {user_id}: {e}")

# Сохраняем с явным преобразованием типов
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(predictions, f, indent=2, ensure_ascii=False, default=str)

In [ ]:
import pandas as pd

# Таблица с топ-рекомендациями
df_results = pd.DataFrame.from_dict(predictions, orient='index')
print("Топ-рекомендации:")
display(df_results[['top_items', 'predicted_place']].head(10))

# График распределения рекомендуемых товаров
all_items = [item for pred in predictions.values() for item in pred['top_items']]
pd.Series(all_items).value_counts().head(15).plot(kind='barh', title="Самые частые рекомендации");

In [ ]:
# Собираем фактические покупки
actual_purchases = {}
for _, row in df_new.iterrows():
    user_id = str(row['customerId'])
    if user_id not in actual_purchases:
        actual_purchases[user_id] = []
    actual_purchases[user_id].append(row['goodsName'])

# Сравниваем
for user_id, pred in predictions.items():
    print(f"\nПользователь {user_id}:")
    print("Рекомендовано:", pred['top_items'])
    print("Купил:", actual_purchases.get(user_id, "Нет данных"))